# Continual pre-training

En este cuaderno se mostrará como realizar un continual pre-training sobre un gran modelo de lenguaje (LLM), concretamente el modelo GPT-2.

La técnica de continual pre-training consiste en continuar el proceso de entrenamiento de un modelo de lenguaje utilizando nuevos textos en plano. Esta técnica, englobada la técnica de continual learning, se puede utilizar principalmente para dos propósitos:

1. **Adaptación a nuevos dominios**: Si se dispone de un modelo de lenguaje pre-entrenado, se puede continuar el proceso de entrenamiento con textos de un dominio específico para adaptar el modelo a dicho dominio.
2. **Incorporación de nuevas lenguas**: Si se dispone de un modelo de lenguaje pre-entrenado en una lengua, se puede continuar el proceso de entrenamiento con textos en otra lengua para adaptar el modelo a dicha lengua.

En este cuaderno se utilizará la técnica para el segundo de los propósitos. Mediante un proceso simplificado (hacer un ejemplo completo y bien requeriría demasiado tiempo, y en especial, recursos computacionales), se mostrará como continuar el entrenamiento de un modelo GPT-2 pre-entrenado en inglés con textos en valenciano.

## Paso 1: Importación de librerías

Se instalan primero las librerías necesarias.

In [1]:
!pip install transformers==4.27.2
!pip install datasets==2.15.0
!pip install tqdm==4.66.1
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.27.2 which is incompatible.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other require

Se importan las librerías necesarias para realizar el proceso de continual pre-training.

In [2]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, Dataset as HFDataset
from tqdm import tqdm
import random
import numpy as np

## Paso 2: Importación del modelo y del tokenizador

Haciendo uso de la librería `transformers`, se importa el modelo GPT-2 pre-entrenado en inglés y su tokenizador. Se trasladará el modelo a la GPU para acelerar el proceso de entrenamiento.

In [3]:
# Verificación de la GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Se fija una semilla para la reproducibilidad de los resultados
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Usando dispositivo: cuda


In [4]:
# Nombre del modelo a utilizar
model_name = 'gpt2'

# Cargar el tokenizador y el modelo
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Mover el modelo a la GPU
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
#Fijar el token de padding (si no está fijado)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

pad_token_id = tokenizer.pad_token_id

## Paso 3: Prueba del modelo antes de continuar el entrenamiento

Antes de realizar el proceso de continual pre-training, se probará el modelo con un simple texto en valenciano para ver que nos responde.

Para generar texto, se ha definido la función `generar_texto` que recibe un texto inicial, un modelo y un tokenizador, y una cantidad máxima de tokens a generar. La función genera texto a partir del texto inicial y el modelo.

In [6]:
# Crear una funcion para generar texto
def generar_texto(model, tokenizer, texto_inicial, max_new_tokens=20):
    inputs = tokenizer.encode(texto_inicial, return_tensors='pt').to(device)
    outputs = model.generate(
        inputs,
        max_length=50,  # Adjust as needed
        num_return_sequences=1,  # Number of sequences to generate
        temperature=1.0,  # Adjust for creativity
        top_k=50,  # Use top-k sampling for better results
        top_p=0.95,  # Use nucleus sampling
        do_sample=True,  # Ensure sampling is enabled for varied output
        pad_token_id=tokenizer.eos_token_id,  # Set the pad token id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
texto_prueba = "Hui parlarem de"
print(generar_texto(model, tokenizer, texto_prueba))

Hui parlarem della nella prolere, nella sua nella storati sua sunt, per parli dixit, per ciusto, per sunt laetitia.




Vemos que el texto generado dista mucho de lo que podría ser valenciano. De hecho, se podría asemejar más a un texto en italiano (sin ser este tampoco correcto).

¿Qué pasará cuando le demos un pequeño corpus en valenciano para continuar el entrenamiento? ¡Vamos a verlo!

## Paso 4: Continuar el entrenamiento

Primero de todo, se realiza la carga del corpus en valenciano. Concretamente, se ha utilizado el corpus `xnli_va`, el cual contiene premisas e hipótesis en valenciano. Como en este cuaderno lo que queremos hacer es un continual pre-training, se ha utilizado solo la columna de la hipótesis.

In [8]:
#Load the gplsi/xnli_va dataset
dataset = load_dataset("gplsi/xnli_va")

#Drop columns hypothesis and label
dataset = dataset.remove_columns(['hypothesis', 'label'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [9]:
import pandas as pd

# Convertir a DataFrame para eliminar duplicados
datasetDf = pd.DataFrame(dataset['test'])  # Cambia 'dataset' por el split específico

# Eliminar duplicados
datasetDf.drop_duplicates(inplace=True)

# Reconstruir el dataset en formato Hugging Face
dataset['test'] = HFDataset.from_pandas(datasetDf).remove_columns(['__index_level_0__'])

Una vez cargado y procesado el corpus, procedemos a tokenizarlo. Para ello, crearemos dos funciones:

- `tokenize_function`: Función que tokeniza un texto.
- `prepare_dataloader`: Función que prepara un DataLoader con los textos tokenizados. Un DataLoader es un objeto que permite iterar sobre los datos de forma eficiente mientras se entrena un modelo.

In [10]:
# Define block size and batch size
block_size = 128
batch_size = 4  # Adjust based on GPU memory

def tokenize_function(examples):
    return tokenizer(examples['premise'], padding='max_length', truncation=True, max_length=block_size)

# Function to prepare DataLoader for a given task and split
def prepare_dataloader(task_dataset):
    # Tokenize the dataset
    tokenized = task_dataset.map(tokenize_function, batched=True)

    # Set 'labels' equal to 'input_ids' for language modeling
    tokenized = tokenized.map(lambda x: {'labels': x['input_ids']}, batched=True)

    # Set format for PyTorch
    tokenized.set_format(type='torch', columns=['input_ids', 'labels'])

    # Create DataLoader
    dataloader = DataLoader(tokenized, shuffle=True, batch_size=batch_size)
    return dataloader


Se aplican ahora todas las funciones definidas sobre el conjunto de prueba, que es el que se utilizará para continuar el entrenamiento (se ha escogido este por ser el de menor tamaño).

In [11]:
dataloader = prepare_dataloader(dataset['test'])

Map:   0%|          | 0/1670 [00:00<?, ? examples/s]

Map:   0%|          | 0/1670 [00:00<?, ? examples/s]

Se define ahora la función que puramente realizará el proceso de continual pre-training:
- `train`: Función que realiza el proceso de continual pre-training. Por cada época, se llamará a esta función, la cuál recorre el DataLoader y entrena el modelo, actualizando los pesos del mismo.

In [12]:
def train(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc="Training")

    for batch in progress_bar:
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        progress_bar.set_postfix({'loss': loss.item()})

    avg_loss = total_loss / len(dataloader)
    return avg_loss

Teniendo ya la función definida, podemos proceder a realizar el entrenamiento. Se definen ciertos parámetros para el entrenamiento, el optimizador y el scheduler.

En este caso se entrenará el modelo usando 2 épocas.

In [13]:
# Parámetros de entrenamiento
epochs = 2
learning_rate = 5e-5
weight_decay = 0.01
warmup_steps = 0

# Optimizador a utilizar
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Directorio para guardar los checkpoints
checkpoint_base_dir = '/content/drive/MyDrive/continual_gpt2_checkpoints'
os.makedirs(checkpoint_base_dir, exist_ok=True)

# Calcular el total de pasos de entrenamiento, que son el número de épocas por el número de lotes
total_steps = epochs * len(dataloader)

# Definir el scheduler, que ajusta la tasa de aprendizaje durante el entrenamiento
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Entrenar el modelo
for epoch in range(epochs):
    print(f"\n--- Epoch {epoch + 1}/{epochs} ---")

    # Entrenar
    avg_train_loss = train(model, dataloader, optimizer, scheduler, device)
    print(f"Average Training Loss: {avg_train_loss}")


# Guardar el modelo y el tokenizador entrenados
checkpoint_dir = os.path.join(checkpoint_base_dir, f"checkpoint")
os.makedirs(checkpoint_dir, exist_ok=True)
model.save_pretrained(checkpoint_dir)
tokenizer.save_pretrained(checkpoint_dir)
print(f"Checkpoint saved at {checkpoint_dir}")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



--- Epoch 1/2 ---


Training: 100%|██████████| 418/418 [01:09<00:00,  6.04it/s, loss=1.13]


Average Training Loss: 1.6227397095928922

--- Epoch 2/2 ---


Training: 100%|██████████| 418/418 [01:13<00:00,  5.70it/s, loss=0.796]


Average Training Loss: 1.3784042426416179
Checkpoint saved at /content/drive/MyDrive/continual_gpt2_checkpoints/checkpoint


## Paso 5: Prueba del modelo después de continuar el entrenamiento

Una vez el modelo, teóricamente, ha aprendido valenciano, procedemos a probarlo con el mismo texto que antes. Veremos si ha mejorado su capacidad de generar texto en valenciano.

In [14]:
texto_prueba = "Hui parlarem de"
print(generar_texto(model, tokenizer, texto_prueba))

Hui parlarem de la mósica ha són o'erment a la seua una potera per a treball i una una seua potera.


Dada la salida generada, podemos ver como, sin ser un valenciano perfecto, la respuesta otorgada ya se asemeja bastante más a un texto en valenciano que la respuesta anterior. Claro está que este texto generado está lejos de poderse considerar como bueno, pero es un buen indicativo de que el modelo ha aprendido algo de valenciano utilizando la técnica de continual pre-training.

Con un corpus más grande y más épocas de entrenamiento, se podría obtener un modelo que genere texto en valenciano de una calidad mucho mayor.